# Moon

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@develop

In [2]:
#%%capture
#!pip install git+https://github.com/fuyu-quant/IBLM.git

In [3]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.2.13


In [4]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [5]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLModel

import os
import sys
sys.path.append('../..')

from src.data.moon import moon_data

In [6]:
df_train, df_test = moon_data(num_train=300, seed = 3658)
df_train.head()

train data:300
test data:700


,Feature_1,Feature_2,target
549,0.311,-0.250,1
531,-0.934,0.297,0
75,0.487,-0.311,1
89,-0.273,0.976,0
585,1.919,0.067,1


In [7]:
x_train = df_train.drop('target', axis=1)
y_train = df_train['target']
print(len(x_train))

300


In [8]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {
    'columns_name': True,
    'objective': 'classification',
    }
iblm = IBLModel(llm_model = llm_model, params = params)

### Training

In [9]:
model = iblm.fit(x_train, y_train)

AttributeError: 'NoneType' object has no attribute 'format'

In [26]:
# Code of the model created
print(model)

import numpy as np
import pandas as pd

def predict(x):
    df = x.copy()
    output = []

    # Calculate the mean and standard deviation of each feature for each target class
    means = df.groupby('target').mean()
    stds = df.groupby('target').std()

    # Calculate the prior probabilities of each target class
    priors = df['target'].value_counts() / len(df)

    for index, row in df.iterrows():
        # Calculate the likelihood of the data given each target class
        likelihood_0 = np.prod(np.exp(-(row[['Feature_1', 'Feature_2']] - means.loc[0])**2 / (2 * stds.loc[0]**2)) / np.sqrt(2 * np.pi * stds.loc[0]**2))
        likelihood_1 = np.prod(np.exp(-(row[['Feature_1', 'Feature_2']] - means.loc[1])**2 / (2 * stds.loc[1]**2)) / np.sqrt(2 * np.pi * stds.loc[1]**2))

        # Calculate the posterior probabilities of each target class
        posterior_0 = likelihood_0 * priors[0]
        posterior_1 = likelihood_1 * priors[1]

        # Normalize the posterior probabilities
  

### Prediction

In [27]:
x_test = df_test.drop('target', axis=1)
y_test = df_test['target']

In [28]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

KeyError: 'target'

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.61375
Precision: 0.5658465991316932
Recall: 0.9775
F1 score: 0.7167736021998167
ROC-AUC: 0.54785625


### Create multiple code models

In [60]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

#### 3655

In [61]:
seed = 3655
n = 300

train_num = [10, 20, 30, 40, 50, 100, 200, 300]
#train_num = [200]

for i in train_num:
    df = pd.read_csv(f'../data/moon/moon_{seed}_{n}_train.csv').head(i)
    x_train = df.drop('target', axis=1)
    y_train = df['target']

    file_path = '../models/moon/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'moon_{seed}_{i}_{j}', file_path=file_path)

Tokens Used: 845
	Prompt Tokens: 477
	Completion Tokens: 368
Successful Requests: 1
Total Cost (USD): $0.03639
Tokens Used: 846
	Prompt Tokens: 477
	Completion Tokens: 369
Successful Requests: 1
Total Cost (USD): $0.036449999999999996
Tokens Used: 909
	Prompt Tokens: 477
	Completion Tokens: 432
Successful Requests: 1
Total Cost (USD): $0.040229999999999995
Tokens Used: 848
	Prompt Tokens: 477
	Completion Tokens: 371
Successful Requests: 1
Total Cost (USD): $0.03657
Tokens Used: 872
	Prompt Tokens: 477
	Completion Tokens: 395
Successful Requests: 1
Total Cost (USD): $0.038009999999999995
Tokens Used: 828
	Prompt Tokens: 477
	Completion Tokens: 351
Successful Requests: 1
Total Cost (USD): $0.03537
Tokens Used: 955
	Prompt Tokens: 477
	Completion Tokens: 478
Successful Requests: 1
Total Cost (USD): $0.04299
Tokens Used: 948
	Prompt Tokens: 477
	Completion Tokens: 471
Successful Requests: 1
Total Cost (USD): $0.04257
Tokens Used: 883
	Prompt Tokens: 477
	Completion Tokens: 406
Successful R

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sun, 02 Jul 2023 10:35:51 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e0636dee88d8a7a-NRT', 'alt-svc': 'h3=":443"; ma=86400'}.


Tokens Used: 992
	Prompt Tokens: 601
	Completion Tokens: 391
Successful Requests: 1
Total Cost (USD): $0.04149
Tokens Used: 1224
	Prompt Tokens: 601
	Completion Tokens: 623
Successful Requests: 1
Total Cost (USD): $0.055409999999999994
Tokens Used: 1000
	Prompt Tokens: 601
	Completion Tokens: 399
Successful Requests: 1
Total Cost (USD): $0.04196999999999999
Tokens Used: 1002
	Prompt Tokens: 601
	Completion Tokens: 401
Successful Requests: 1
Total Cost (USD): $0.042089999999999995
Tokens Used: 1095
	Prompt Tokens: 601
	Completion Tokens: 494
Successful Requests: 1
Total Cost (USD): $0.04767
Tokens Used: 1122
	Prompt Tokens: 601
	Completion Tokens: 521
Successful Requests: 1
Total Cost (USD): $0.049289999999999994
Tokens Used: 1125
	Prompt Tokens: 601
	Completion Tokens: 524
Successful Requests: 1
Total Cost (USD): $0.04946999999999999
Tokens Used: 1122
	Prompt Tokens: 601
	Completion Tokens: 521
Successful Requests: 1
Total Cost (USD): $0.049289999999999994
Tokens Used: 1015
	Prompt Tok

#### 3656

In [ ]:
seed = 3656
n = 300

train_num = [10, 20, 30, 40, 50, 100, 200, 300]
#train_num = [200]

for i in train_num:
    df = pd.read_csv(f'../data/moon/moon_{seed}_{n}_train.csv').head(i)
    x_train = df.drop('target', axis=1)
    y_train = df['target']

    file_path = '../models/moon/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'moon_{seed}_{i}_{j}', file_path=file_path)

#### 3657

In [ ]:
seed = 3657
n = 300

train_num = [10, 20, 30, 40, 50, 100, 200, 300]
#train_num = [200]

for i in train_num:
    df = pd.read_csv(f'../data/moon/moon_{seed}_{n}_train.csv').head(i)
    x_train = df.drop('target', axis=1)
    y_train = df['target']

    file_path = '../models/moon/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'moon_{seed}_{i}_{j}', file_path=file_path)

### Models Evaluation

In [ ]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [ ]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

In [125]:
seed = 3655
#seed = 3656
#seed = 3657

# 使うcsvファイルのデータ数
n = 300
train_num = [10, 20, 30, 40, 50, 100, 200, 300]
i = 10

df = pd.read_csv(f'../data/moon/moon_{seed}_{n}_test.csv')
x_test = df.drop('target', axis=1)
y_test = df['target']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []
acc_list = []
model_list = []

for j in range(1,31):

    import_file = f'import models.moon.moon_{seed}_{i}_{j} as codemodel'

    try:
        exec(import_file)
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba <= 0)
        values_greater_than_one_exist = np.any(y_proba >= 1)
        if negative_values_exist:
            error_list.append(j)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(j)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            accuracy = round(accuracy_score(y_test, y_pred),4)
            auc_list.append(roc_auc)
            acc_list.append(accuracy)
            model_list.append(f'moon_{seed}_{i}_{j}')
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(j)
        pass

print('-----------error-------------')
print(error_count)
print(error_list)

print('-----------Model-------------')
print(model_list)

print('-----------ACC-------------')
print(acc_list)
average = sum(acc_list) / len(acc_list)
max_value = max(acc_list)
min_value = min(acc_list)
print("Maximum Value:", max_value)
print("Average Value:", round(average,4))
print("Minimum Value:", min_value)

print('-----------AUC-------------')
print(auc_list)
average = sum(auc_list) / len(auc_list)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", round(max_value,4))
print("Average Value:", round(average,4))
print("Minimum Value:", round(min_value,4))

Negative values exist：True
run error
run error
run error
Negative values exist：True
Negative values exist：True
Negative values exist：True
run error
Negative values exist：True
Negative values exist：True
Negative values exist：True
Negative values exist：True
run error
run error
run error
Negative values exist：True
Positive values exist：True
run error
Negative values exist：True
run error
run error
Negative values exist：True
Negative values exist：True
Negative values exist：True
-----------error-------------
24
[1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 28, 29, 30]
-----------Model-------------
['moon_3655_10_2', 'moon_3655_10_8', 'moon_3655_10_18', 'moon_3655_10_19', 'moon_3655_10_21', 'moon_3655_10_27']
-----------AUC-------------
[0.3410222222222222, 0.9577777777777778, 0.8233333333333333, 0.9577777777777778, 0.9577777777777778, 0.8233333333333333]
Maximum Value: 0.9578
Average Value: 0.8102
Minimum Value: 0.341
-----------ACC-------------
[0.3067, 0.853